In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


def preprocess_text(text, stop_words):
    # Remove special characters and lowercase the text
    text = re.sub(r"[^\w\s]", "", text.lower())
    # Remove stopwords
    text = " ".join(word for word in text.split() if word not in stop_words)
    return text


def build_model(vocab_size):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=vocab_size))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))
    return model


def train_model(model, X_train, y_train, X_val, y_val, epochs=20, batch_size=32):
    print('X_train shape :', X_train.shape)
    print('X_test shape :' ,X_val.shape)
    print('y_train shape :' ,y_val.shape)
    max
    early_stop = EarlyStopping(monitor='val_loss', patience=3)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), callbacks=[early_stop])


def evaluate_model(model, X_train, y_train, X_test, y_test):
    train_loss, train_acc = model.evaluate(X_train, y_train)
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print("Train Loss:", train_loss)
    print("Train Accuracy:", train_acc)
    print("Test Loss:", test_loss)
    print("Test Accuracy:", test_acc)


def text_emotion_detection(dataset_path, stopwords_path):
    # Step 1: Load the dataset
    data = pd.read_csv(dataset_path)

    # Step 2: Load stopwords
    stop_words = set(open(stopwords_path, 'r').read().split())

    # Step 3: Preprocess the text
    data['Text'] = data['Text'].apply(lambda x: preprocess_text(x, stop_words))

    # Step 4: Split the dataset into train, validation, and test sets
    train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
    train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

    # Step 5: Tokenize the text
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(data['Text'])
    train_data_encoded = tokenizer.texts_to_matrix(train_data['Text'], mode='binary')
    val_data_encoded = tokenizer.texts_to_matrix(val_data['Text'], mode='binary')
    test_data_encoded = tokenizer.texts_to_matrix(test_data['Text'], mode='binary')

    # Step 6: Encode emotion labels
    label_encoder = LabelEncoder()
    label_encoder.fit(train_data['Label'])
    train_labels_encoded = label_encoder.transform(train_data['Label'])
    val_labels_encoded = label_encoder.transform(val_data['Label'])
    test_labels_encoded = label_encoder.transform(test_data['Label'])

    # Step 7: Build the model
    vocab_size = len(tokenizer.word_index) + 1
    model = build_model(vocab_size)

    # Step 8: Train the model
    train_model(model, train_data_encoded, train_labels_encoded, val_data_encoded, val_labels_encoded)

    # Step 9: Evaluate the model
    evaluate_model(model, train_data_encoded, train_labels_encoded, test_data_encoded, test_labels_encoded)


# Example usage
dataset_path = 'Dataset/Text_Emotion_Data.csv'
stopwords_path = 'Dataset/stopwords.txt'
text_emotion_detection(dataset_path, stopwords_path)


Epoch 1/20
65/65 [==============================] - 1s 6ms/step - loss: 1.6074 - accuracy: 0.2337 - val_loss: 1.5967 - val_accuracy: 0.3308
Epoch 2/20
65/65 [==============================] - 0s 3ms/step - loss: 1.5654 - accuracy: 0.3303 - val_loss: 1.5492 - val_accuracy: 0.4808
Epoch 3/20
65/65 [==============================] - 0s 3ms/step - loss: 1.4170 - accuracy: 0.4837 - val_loss: 1.3619 - val_accuracy: 0.6154
Epoch 4/20
65/65 [==============================] - 0s 3ms/step - loss: 1.1065 - accuracy: 0.6264 - val_loss: 1.0847 - val_accuracy: 0.7077
Epoch 5/20
65/65 [==============================] - 0s 3ms/step - loss: 0.7874 - accuracy: 0.7423 - val_loss: 0.8697 - val_accuracy: 0.7442
Epoch 6/20
65/65 [==============================] - 0s 3ms/step - loss: 0.5616 - accuracy: 0.8231 - val_loss: 0.7451 - val_accuracy: 0.7769
Epoch 7/20
65/65 [==============================] - 0s 3ms/step - loss: 0.4108 - accuracy: 0.8913 - val_loss: 0.6775 - val_accuracy: 0.7769
Epoch 8/20
65/65 [==